In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
%matplotlib tk

# Cálculo de Blindajes

### histograma

In [2]:
materiales = ['Uranio', "Wolframio", "Plomo", "Hierro", "Concreto"]

# Numero atomico
Z = np.array([92,74, 82, 26, 11])
# Densidades (g/cm³)
densidad = np.array([18.9, 19.3, 11.35, 7.87, 2.3])



fig, ax = plt.subplots(1, 2, figsize=(12, 4))
for i in range(len(materiales)):
    ax[0].bar(materiales[i], Z[i],width=0.6)
    ax[1].bar(materiales[i], densidad[i],width=0.6)

ax[0].set_title(r"Número atómico Z",fontsize=18)
ax[0].set_ylabel(r"Z",fontsize=18)
  
ax[1].set_title("Densidad",fontsize=18)
ax[1].set_ylabel("g/cm³",fontsize=18)

for i in range(2):
    ax[i].tick_params(axis='y', labelsize=16)
    ax[i].tick_params(axis='x', labelsize=16)

    
plt.tight_layout()
plt.show()
fig.savefig('plots/hist_Z_rho.png',dpi=300)

In [3]:
materiales = ['Polietileno', 'Agua']

# Numero atomico
Z = np.array([5.5,7.4])
# Densidades (g/cm³)
densidad = np.array([0.95, 1])
# Concentracion (atomos/cm³)
x = np.array([8, 6.7])


fig, ax = plt.subplots(1, 3, figsize=(12, 4))
for i in range(len(materiales)):
    ax[0].bar(materiales[i], Z[i],width=0.6)
    ax[1].bar(materiales[i], densidad[i],width=0.6)
    ax[2].bar(materiales[i], x[i],width=0.6)

ax[0].set_title(r"Número atómico Z",fontsize=18)
ax[0].set_ylabel(r"Z",fontsize=18)
  
ax[1].set_title("Densidad",fontsize=18)
ax[1].set_ylabel("g/cm³",fontsize=18)

ax[2].set_title(r"$X_{H}\times 10^{22}$",fontsize=18)
ax[2].set_ylabel("átomos/cm³",fontsize=18)

for i in range(3):
    ax[i].tick_params(axis='y', labelsize=16)
    ax[i].tick_params(axis='x', labelsize=16)

    
plt.tight_layout()
plt.show()
fig.savefig('plots/hist_z_rho_v2.png',dpi=300)

# Metodología

In [2]:
import numpy
import math

class Blindaje:
    def __init__(self, tau, gamma, A0, N, d):
        """Parámetros necesarios para los cálculos."""
        self.tau = tau      # vida media
        self.gamma = gamma  # constante gamma
        self.A0 = A0        # actividad inicial
        self.N = N          # número de pacientes/semana
        self.d = d          # distancia

    def f_decaimiento(self, t):
        """Calcula el factor de decaimiento fd"""
        return np.exp(-np.log(2) * t / self.tau)
    
    def f_reduccion(self, t):
        """Calcula el factor de reduccion Rt"""
        #return self.tau/(t*np.log(2))*(1-np.exp(-t/self.tau*np.log(2)))
        return self.tau/(t*np.log(2))*(1-self.f_decaimiento(t))
    
    def dosis(self, t):
        """Dosis sin considerar blindaje"""
        fd = self.f_decaimiento(t)
        Rt = self.f_reduccion(t)
        return self.gamma*self.A0*self.N*t*fd*Rt/(self.d)**2
    
    

### Inicio

In [3]:
from parameters import presets
sel = presets["Tc99m"]
B = Blindaje(tau = sel["tau"], gamma = sel["gamma"],
                 A0 = sel["A0"], N = sel["N"], d = sel["d"])


### Factor de decaimiento

In [4]:
from parameters import presets
sel = presets["Tc99m"]
B = Blindaje(tau = sel["tau"], gamma = sel["gamma"],
                 A0 = sel["A0"], N = sel["N"], d = sel["d"])

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
time = np.linspace(1e-3,5*sel['tau'],51)
ax.plot(time, B.f_decaimiento(time), 's')
label = r'$f_d(t)=e^{-t/\tau \ln(2)}$, $\tau = 6$h'
ax.plot(time, B.f_decaimiento(time), '-',
        label=label, lw=2)

ax.set_xlabel('Tiempo [h]',fontsize=16)
ax.set_ylabel(r'$f_d$', fontsize=18)

ax.tick_params(axis='y', labelsize=16)
ax.tick_params(axis='x', labelsize=16)
ax.legend(fontsize=16)
ax.grid(True)

#ax.set_title('Factor de decaimiento del Tc-99m',fontsize=20)

plt.tight_layout()
plt.show()
fig.savefig('plots/fd.png',dpi=300)


## factor de reduccion

In [4]:
from parameters import presets
sel = presets["Tc99m"]
B = Blindaje(tau = sel["tau"], gamma = sel["gamma"],
                 A0 = sel["A0"], N = sel["N"], d = sel["d"])


In [55]:

fig, ax = plt.subplots(1,3,figsize=(14,4))
time = np.linspace(1e-3,0.1*sel['tau'],51)
ax[0].plot(time/sel['tau'], B.f_reduccion(time), 's')

time = np.linspace(1e-3,sel['tau'],51)
ax[1].plot(time/sel['tau'], B.f_reduccion(time), 's')

time = np.linspace(1e-3,20*sel['tau'],51)
ax[2].plot(time/sel['tau'], B.f_reduccion(time), 's')
#ax[2].plot(time/sel['tau'], B.f_reduccion(time), '-')

# label = r'$f_d(t)=e^{-t/\tau \ln(2)}$, $\tau = 6$h'
# ax.plot(time/sel['tau'], B.f_reduccion(time), '-',
#         label=label, lw=2)
for i in range(3):
    ax[i].set_xlabel(r'$t/\tau$',fontsize=16)
    ax[i].set_ylabel(r'$R_t$', fontsize=18)
    ax[i].tick_params(axis='x', labelsize=16)
    ax[i].tick_params(axis='y', labelsize=16)
    ax[i].grid(True)
ax[0].set_title(r'$t\ll \tau$, $R_t \approx 1$',fontsize=18)
ax[1].set_title(r'$t\sim \tau$, $R_t < 1$',fontsize=18)
ax[2].set_title(r'$t\gg \tau$',fontsize=18)

# ax[0].annotate(r'$R_t \approx 1$',xy=(0.075,0.995),fontsize=16)
# ax[1].annotate(r'$R_t < 1$',xy=(0.75,0.95),fontsize=16)

label = r'$R_t = t/(\tau \ln(2))$'
ax[2].hlines(y = sel['tau']/(time[-1]*np.log(2)),
    xmin = time.min()/sel['tau'],
    xmax = time.max()/sel['tau'], color='red',lw=2,
             label = label
)


ax[2].legend(fontsize=16)


#ax.set_title('Factor de decaimiento del Tc-99m',fontsize=20)

plt.tight_layout()
plt.show()
fig.savefig('plots/Rt.png',dpi=300)


In [6]:
from parameters import presets2
sel = presets2["Hierro"]


In [7]:
import numpy
import math

class Espesor:
    def __init__(self, alpha,beta,gamma):
        """Parámetros necesarios para los cálculos."""
        self.alpha = alpha     
        self.beta = beta  
        self.gamma = gamma        

    def factor_B(self, P,D,T):
        """Calcula el factor de transmision B"""
        return P/(D*T)
    
    def d_espesor(self, B):
        """Calcula el espesor X"""
        c = self.beta/self.alpha
        cociente = (B**(-self.gamma)+c)/(1+c)
        X = (1/ self.alpha * self.gamma ) * np.log(cociente)
        return X
    
    

In [8]:
from parameters import presets2
sel = presets2["Plomo"]
X = Espesor(alpha = sel["alpha"], beta = sel["beta"],
            gamma = sel["gamma"])


In [16]:
fig,ax = plt.subplots()
B_values = np.linspace(1e-3,1,51)
ax.plot(B_values, X.d_espesor(B_values), 's')
ax.set_xlabel('Factor de Transmision',fontsize=16)
ax.set_ylabel('Espesor [m]',fontsize=16)

ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
ax.grid(True)
fig.savefig('plots/espesor.png',dpi=300)

In [12]:
def calcular_D_1(t, tao, gamma, A0, N, d):
    fd = e**(-np.log(2)*t/tao)
    
    Rt = 1.44*tao/t*(1-e**(-0.693*t/tao))
    D = gamma*A0*N*t*fd*Rt/d**2
    
    return "DOSIS SIN BLINDAJE EN EL AREA OCUPADA --> D = " + str(D)

#Llamada a la funcion:
calcular_D_1(1,2,3,4,5,3)

'DOSIS SIN BLINDAJE EN EL AREA OCUPADA --> D = 3.975743706098797'

In [14]:
def calcular_D_2():
    
    t = float(input("Ingrese el tiempo t: "))
    print("Seleccione el tiempo de vida media")
    valor = input("a.Tecnecio, b.Fluor , x.Otro : ")
    
    if valor == 'a':
        tao = 6
    elif valor == 'b':
        tao = 1.83
    else :
        tao = float(input("Ingrese el tao en horas: "))
        
    gam = float(input("Ingrese el numero gamma en (uSvm**2/MBq.h): "))
    A0 = float(input("Ingrese el A0 en (MBq): "))
    N = float(input("Pacientes maximos por semana N en (1/sem): "))
    d = float(input("Distancia d en (m**2): "))
    
    Fd = e**(-0.693*t/tao)
    print("      Fd = " + str(Fd))
    
    Rt = (1.44*tao/t)*(1-Fd)
    print("      Rt = " + str(Rt))
    
    D = gam*A0*N*t*Fd*Rt/d**2
    return "DOSIS SIN BLINDAJE EN EL AREA OCUPADA --> D = " + str(D) + "uSv/sem"

#Llamada a la funcion:
calcular_D_2()


Ingrese el tiempo t:  1


Seleccione el tiempo de vida media


a.Tecnecio, b.Fluor , x.Otro :  x
Ingrese el tao en horas:  2
Ingrese el numero gamma en (uSvm**2/MBq.h):  3
Ingrese el A0 en (MBq):  4
Pacientes maximos por semana N en (1/sem):  5
Distancia d en (m**2):  7


      Fd = 0.7071588192872713
      Rt = 0.8433826004526586


'DOSIS SIN BLINDAJE EN EL AREA OCUPADA --> D = 0.7302923803390169uSv/sem'

In [15]:
def calcular_B_1(P, T, D):
    B = P/ D*T
    return "FACTOR DE TRANSMISION -->  B = " + str(B)

#Llamada a la funcion:
calcular_B_1(43.5,34, 34)

'FACTOR DE TRANSMISION -->  B = 43.5'

In [16]:
def calcular_B_2():
    
    P = float(input("Ingrese el dato P: "))
    T = float(input("Ingrese el dato T: "))
    D = float(input("Ingrese el dato D: "))
    
    B = P/(D*T)
    return "FACTOR DE TRANSMISION -->  B = " + str(B) 
  
#Llamada a la funcion:
calcular_B_2()

Ingrese el dato P:  1
Ingrese el dato T:  2
Ingrese el dato D:  3


'FACTOR DE TRANSMISION -->  B = 0.16666666666666666'

In [19]:
def calcular_X_1(alpha, beta, Gamma, B):
    
    constante = beta /alpha
    cociente = ((B**(-Gamma)) +constante)/(1+ constante)
    X = (1/alpha*Gamma)*np.log(cociente,e)
    
    return "ESPESOR DEL BLINDAJE --> X =  " + str(X) 
  
#Llamada a la funcion
calcular_X_1(34,243,232, 454.34)


TypeError: return arrays must be of ArrayType

In [20]:
def calcular_X_2():

    print("Seleccione los parametros de transmision")
    dato = input("a.Lead, b.Concrete , c.Iron, x.Otros : ")
    if dato == 'a':
        alpha = 1.543
        beta = -0.4408
        Gam = 2.136
    elif dato == 'b':
        alpha = 0.1539
        beta = -0.1161
        Gam = 2.0752
    elif dato == 'c' :
        alpha = 0.5704
        beta = -0.3063
        Gam = 0.6326
    else :
        alpha = float(input("Ingrese alpha en (1/cm) : "))
        beta = float(input("Ingrese beta en (1/cm) : "))
        Gam = float(input("Ingrese Gamma : "))
    
    B = float(input("Ingrese el factor de transmision B : "))
    
    #Usamos la ecuacion de Acher:
    c = beta /alpha
    
    cociente = ((B**(-Gam)) + c)/(1+ c)
    
    X = (1/alpha*Gam)*log(cociente,e)
    
    #Para mayor comodidad convertimos el X calculado con valores del Plomo a mm.
    '''
    Mediante condicionales comprobamos que si en el caso de haberse elegido los parametros de transmision del Plomo, se devidira entre diez el X calculado para obtenerlo en mm.
    '''
    
    if dato == 'a':   
        X = X/10
        Y = 'mm'
    else :
        X = X
        Y = 'cm'
    
    return "ESPESOR DEL BLINDAJE --> X =  " + str(X) +  Y 

#Llamada  a la funcion
calcular_X_2()



Seleccione los parametros de transmision


a.Lead, b.Concrete , c.Iron, x.Otros :  a
Ingrese el factor de transmision B :  1


NameError: name 'log' is not defined

In [21]:
def Calculo_secuencial_de_todo():
    #El codigo solicita de manera secuecial las variables que se usaran para 
    
    #1
    t = float(input("Tiempo de permanencia en horas t : "))
    valor = input("a.Tecnecio, b.Fluor : ")
    if valor == 'a':
        tao = 6
    elif valor == 'b':
        tao = 1.83
    else :
        tao = float(input("Ingrese el tiempo medio de vida: "))
    gam = float(input("Ingrese el numero gamma: "))
    A0 = float(input("Ingrese el A0: "))
    N = float(input("Pacientes maximos por semana N: "))
    d = float(input("Distancia d: "))
    
    #Factor de decaimiento del radifarmaco:
    Fd = e**(-0.693*t/tao)
    print("El numero Fd es : " + str(Fd))
    
    #Factor de reduccion:
    Rt = (1.44*tao/t)*(1-Fd)
    print("El numero Rt es: " + str(Rt))
    
    #Dosis sin blindaje en el area ocupada:
    D = gam*A0*N*t*Fd*Rt/d**2
    print("DOSIS SIN BLINDAJE EN EL AREA OCUPADA --> D = " + str(D))
    
    #2
    P = float(input("Restriccion de dosis P: "))
    T = float(input("Factor de ocupacion T: "))
    
    #Factor de transmision:
    B = P/(D*T)
    print("FACTOR DE TRANSMISION -->  B = " + str(B))
    
    #3
    #Calculamos el espesor:
    dato = input("a.Lead, b.Concrete , c.Iron : ")
    if dato == 'a':
        alpha = 1.543
        beta = -0.4408
        Gam = 2.136
    elif dato == 'b':
        alpha = 0.1539
        beta = -0.1161
        Gam = 2.0752
    elif dato == 'c' :
        alpha = 0.5704
        beta = -0.3063
        Gam = 0.6326
    else :
        alpha = float(input("Ingrese alpha: "))
        beta = float(input("Ingrese beta: "))
        Gam = float(input("Ingrese Gamma: "))
   
    #Usamos la ecuacion de Acher:
    c = beta /alpha
    cociente = ((B**(-Gam)) + c)/(1+ c)
    X = (1/alpha*Gam)*ma.log(cociente,e)
    
    return "El numero X es: " + str(X) 

#Llamada a la funcion:
Calculo_secuencial_de_todo()


Tiempo de permanencia en horas t :  1
a.Tecnecio, b.Fluor :  a
Ingrese el numero gamma:  1
Ingrese el A0:  12
Pacientes maximos por semana N:  1
Distancia d:  2


El numero Fd es : 0.8909205722370781
El numero Rt es: 0.9424462558716453
DOSIS SIN BLINDAJE EN EL AREA OCUPADA --> D = 2.518934272751574


Restriccion de dosis P:  1
Factor de ocupacion T:  1


FACTOR DE TRANSMISION -->  B = 0.39699328831936675


a.Lead, b.Concrete , c.Iron :  a


NameError: name 'ma' is not defined

### Energies

In [3]:
import pandas as pd

/home/kelvin/anaconda3/envs/kelvin/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
df=pd.read_csv('data2.txt',sep='\t')
df

,T,alpha5,beta5,gamma5,chi2_5,alpha6,beta6,gamma6,chi2_6
0,15,11.285300,-0.503702,2.066840,0.00010,48.196000,-1.919700,0.604130,2.947700e-03
1,20,9.509950,-0.414115,0.619699,0.00040,38.657000,-7.454400,0.667400,0.000000e+00
2,25,5.321160,-1.735640,1.503320,0.01710,22.266000,-5.893600,1.129700,7.976600e-12
3,30,3.297090,-1.426710,2.771630,0.05490,13.718000,-7.847600,4.403200,2.468000e-02
4,35,2.180410,-1.212940,5.059800,0.06700,9.107200,-3.356700,2.733900,2.304700e-02
5,40,1.530130,-0.507678,2.663230,0.11290,6.374800,-2.883400,4.212500,3.645900e-01
6,45,1.113240,-0.465410,4.358910,0.87230,4.652600,-2.009500,4.402100,6.004700e-01
7,50,0.843768,-0.208544,2.137840,0.01310,3.510000,-1.591600,5.395500,2.180600e+00
8,55,0.651233,-0.180282,3.020430,1.66120,2.197500,-0.127240,0.397630,3.271500e-03
9,60,0.518569,-0.100439,1.748040,0.21460,2.176100,-0.397750,1.550200,7.771100e-01


### Database

In [2]:
import sqlite3

creamos la database

In [3]:
conn = sqlite3.connect("data_new.db")
c = conn.cursor()

creamos la table

In [4]:
c.execute('''
CREATE TABLE IF NOT EXISTS properties (
    material TEXT,
    energy REAL,
    alpha REAL,
    beta REAL,
    gamma REAL,
    chi2 REAL
)
''')

In [5]:
data = [
    ('Acero',10,35.068,80.286,0.1718,0.2623),
    ('Acero',15,42.896,-14.135,1.1513,0.0),
    ('Acero',20,19.721,-9.6886,2.7227,0.00058279),
    ('Acero',25,10.357,-4.9682,3.419,0.004177),
    ('Acero',30,6.1395,-2.0142,2.3201,3.8275e-05),
    ('Acero',35,3.9145,-1.4049,2.9659,0.02918),
    ('Acero',40,2.6645,-0.60655,1.7984,7.1274e-05),
    ('Acero',45,1.8981,-0.40604,1.7432,0.021919),
    ('Acero',50,1.4112,-0.22685,1.1963,0.0011541),
    ('Acero',55,1.08,-0.17254,1.2441,0.0041784),
    ('Acero',60,0.85248,-0.13327,1.1595,0.017991),
    ('Acero',65,0.68752,-0.10514,1.157,0.022259),
    ('Acero',70,0.56286,-0.089759,1.3976,0.010666),
    ('Acero',75,0.47528,-0.075517,1.1653,0.017669),
    ('Acero',80,0.40599,-0.06599,1.1244,1.923),
    ('Acero',85,0.40988,-0.065411,0.88713,0.038793),
    ('Acero',90,0.30668,-0.052336,1.2367,0.0088909),
    ('Acero',95,0.27828,-0.050199,0.83917,0.010167),
    ('Acero',100,0.24895,-0.047754,0.93083,0.015697),
    ('Acero',105,0.22397,-0.044848,1.0666,0.0069165),
    ('Acero',110,0.20743,-0.043758,0.8825,0.013961),
    ('Acero',115,0.18986,-0.041533,0.96743,0.0070746),
    ('Acero',120,0.1761,-0.040423,0.9852,0.004907),
    ('Acero',125,0.16292,-0.038763,1.10821,0.0131796),
    ('Acero',130,0.155,-0.038923,0.96772,0.0046771),
    ('Acero',135,0.014614,-0.037907,0.98233,0.0035513),
    ('Acero',140,0.13877,-0.037207,0.97552,0.00244),
    ('Acero',145,0.13227,-0.036639,0.97319,0.0044865),
    ('Acero',150,0.1265,-0.036001,0.97308,0.0052436)
]

c.executemany('INSERT INTO properties VALUES (?,?,?,?,?,?)', data)



In [6]:
data = [

    ('Concreto',20,8.1578,-0.32627,0.48811,0.0005218),
    ('Concreto',25,4.2935,-0.25526,0.45225,0.0021134),
    ('Concreto',30,2.6098,-0.21955,0.48547,0.0071091),
    ('Concreto',35,1.7272,-0.18611,0.69391,0.00029066),
    ('Concreto',40,1.2587,-0.17887,0.66481,0.00030635),
    ('Concreto',45,0.97031,-0.17347,0.70411,5.7218e-05),
    ('Concreto',50,0.7897,-0.17062,0.70111,0.00014021),
    ('Concreto',55,0.65535,-0.17667,1.0088,0.83235),
    ('Concreto',60,0.57874,-0.16434,0.76275,0.00011372),
    ('Concreto',65,0.51644,-0.15865,0.77217,0.00063926),
    ('Concreto',70,0.43336,-0.15337,0.80137,0.0012265),
    ('Concreto',75,0.43336,-0.15337,0.80137,0.0012265),
    ('Concreto',80,0.40509,-0.15113,0.81031,0.0031039),
    ('Concreto',85,0.38304,-0.14556,0.79734,0.0055415),
    ('Concreto',90,0.36398,-0.14196,0.80786,0.0046497),
    ('Concreto',95,0.35072,-0.14092,0.78707,0.0042387),
    ('Concreto',100,0.33873,-0.1389,0.77229,0.010394),
    ('Concreto',105,0.32506,-0.13365,0.80501,0.0055723),
    ('Concreto',110,0.3154,-0.13042,0.79705,0.0061974),
    ('Concreto',115,0.30864,-0.12954,0.77753,0.0060525),
    ('Concreto',120,0.30056,-0.12694,0.78397,0.0056804),
    ('Concreto',125,0.29464,-0.12547,0.77387,0.0052431),
    ('Concreto',135,0.28188,-0.12023,0.78438,0.0041124),
    ('Concreto',140,0.27729,-0.11859,0.7777,0.0042806),
    ('Concreto',145,0.27221,-0.1165,0.78544,0.0040345),
    ('Concreto',150,0.26821,-0.11491,0.78312,0.0038982)
]

c.executemany('INSERT INTO properties VALUES (?,?,?,?,?,?)', data)




In [7]:
data = [
    ('Plomo',15,11.2853,-0.503702,2.06684,0.0001),
    ('Plomo',20,9.50995,-0.414115,0.619699,0.0004),
    ('Plomo',25,5.32116,-1.73564,1.50332,0.0171),
    ('Plomo',30,3.29709,-1.42671,2.77163,0.0549),
    ('Plomo',35,2.18041,-1.21294,5.0598,0.067),
    ('Plomo',40,1.53013,-0.507678,2.66323,0.1129),
    ('Plomo',45,1.11324,-0.46541,4.35891,0.8723),
    ('Plomo',50,0.843768,-0.208544,2.13784,0.0131),
    ('Plomo',55,0.651233,-0.180282,3.02043,1.6612),
    ('Plomo',60,0.518569,-0.100439,1.74804,0.2146),
    ('Plomo',65,0.418992,-0.0624265,1.30914,0.2838),
    ('Plomo',70,0.34452,-0.0429512,1.01129,0.2614),
    ('Plomo',75,0.285896,-0.0354312,1.16484,0.1981),
    ('Plomo',80,0.241339,-0.0274606,1.02199,0.2),
    ('Plomo',85,0.205821,-0.0216135,0.894837,0.1795),
    ('Plomo',88,0.187505,-0.0195533,0.953233,0.1445),
    ('Plomo',89,0.184312,0.356167,0.213962,3.0177),
    ('Plomo',90,0.174093,0.351449,0.189643,0.27438),
    ('Plomo',95,0.0692092,0.408605,0.0985324,2.217),
    ('Plomo',100,-5.51428,5.94959,0.00378136,2.7667),
    ('Plomo',105,-8.59998,9.00411,0.00174954,6.5412),
    ('Plomo',110,0.0485351,0.31229,0.0137629,3.3735),
    ('Plomo',115,0.337038,-0.156541,12.8629,0.4278),
    ('Plomo',120,0.316073,-0.10148,6.17966,1.9958),
    ('Plomo',125,0.298059,-0.0781643,3.21266,2.9783),
    ('Plomo',130,0.282326,-0.0605881,1.53607,0.6197),
    ('Plomo',135,0.26601,-0.0530182,1.03209,0.1146),
    ('Plomo',140,0.248499,-0.048634,0.903917,0.1281),
    ('Plomo',145,0.23097,-0.0447229,0.883316,0.1838),
    ('Plomo',150,0.212649,-0.0418441,1.09587,0.0984)
]

c.executemany('INSERT INTO properties VALUES (?,?,?,?,?,?)', data)

conn.commit()
conn.close()



Read database

In [8]:
import sqlite3
import pandas as pd

/home/kelvin/anaconda3/envs/kelvin/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [23]:
# Example: select alpha, beta, gamma for Lead at energy=140
conn = sqlite3.connect("data_new.db")
df = pd.read_sql_query(
    "SELECT energy, material, alpha, beta, gamma, chi2 FROM properties WHERE energy = 50",
    conn
)

print(df)

conn.close()


   energy  material     alpha      beta    gamma      chi2
0    50.0     Acero  1.411200 -0.226850  1.19630  0.001154
1    50.0  Concreto  0.789700 -0.170620  0.70111  0.000140
2    50.0     Plomo  0.843768 -0.208544  2.13784  0.013100


In [37]:
# Example: select alpha, beta, gamma for Lead at energy=140
conn = sqlite3.connect("data.db")
df = pd.read_sql_query(
    "SELECT energy, alpha, beta, gamma FROM properties WHERE material='Concreto'",
    conn
)

print(df)

conn.close()


    energy    alpha     beta    gamma
0     20.0  8.15780 -0.32627  0.48811
1     25.0  4.29350 -0.25526  0.45225
2     30.0  2.60980 -0.21955  0.48547
3     35.0  1.72720 -0.18611  0.69391
4     40.0  1.25870 -0.17887  0.66481
5     45.0  0.97031 -0.17347  0.70411
6     50.0  0.78970 -0.17062  0.70111
7     55.0  0.65535 -0.17667  1.00880
8     60.0  0.57874 -0.16434  0.76275
9     65.0  0.51644 -0.15865  0.77217
10    70.0  0.43336 -0.15337  0.80137
11    75.0  0.43336 -0.15337  0.80137
12    80.0  0.40509 -0.15113  0.81031
13    85.0  0.38304 -0.14556  0.79734
14    90.0  0.36398 -0.14196  0.80786
15    95.0  0.35072 -0.14092  0.78707
16   100.0  0.33873 -0.13890  0.77229
17   105.0  0.32506 -0.13365  0.80501
18   110.0  0.31540 -0.13042  0.79705
19   115.0  0.30864 -0.12954  0.77753
20   120.0  0.30056 -0.12694  0.78397
21   125.0  0.29464 -0.12547  0.77387
22   135.0  0.28188 -0.12023  0.78438
23   140.0  0.27729 -0.11859  0.77770
24   145.0  0.27221 -0.11650  0.78544
25   150.0  

### Grafiquemos

In [24]:
conn = sqlite3.connect("data.db")
df = pd.read_sql_query(
    "SELECT energy, alpha, beta, gamma FROM properties WHERE material='Concreto'",
    conn
)

energy = df['energy'].to_numpy()
alpha = df['alpha'].to_numpy()
beta = df['beta'].to_numpy()
gamma = df['gamma'].to_numpy()

conn.close()

In [25]:
fig,ax = plt.subplots(1,3,figsize=(14,3))
ax[0].plot(energy,alpha, '-s')
ax[1].plot(energy,beta, '-s')
ax[2].plot(energy,gamma, '-s')

fontsize=14
for i in range(0,3):
    ax[i].set_xlabel('E [keV]',fontsize=fontsize)
    ax[i].tick_params(axis='x', labelsize=14)
    ax[i].tick_params(axis='y', labelsize=14)
    ax[i].grid(True)
ax[0].set_ylabel(r'$\alpha$',fontsize=fontsize)
ax[1].set_ylabel(r'$\beta$',fontsize=fontsize)
ax[2].set_ylabel(r'$\gamma$',fontsize=fontsize)

color = 'red'
# ax[0].vlines(x=140,ymin=min(alpha), ymax=max(alpha),color=color)
# ax[1].vlines(x=140,ymin=min(beta), ymax=max(beta),color=color)
# ax[2].vlines(x=140,ymin=min(gamma), ymax=max(gamma),color=color)

# for axis, arr in zip([ax[0], ax[1], ax[2]], [alpha, beta, gamma]):
#     axis.annotate(
#         '140 keV', 
#         xy=(140, 1),       # arrow tip
#         xytext=(140, 2),   # start of arrow (slightly above)
#         arrowprops=dict(arrowstyle='->', color=color, lw=2)
#     )

ax[0].annotate(
        '140 keV', 
        xy=(140, 0.5),     
        xytext=(130, 2), 
        arrowprops=dict(arrowstyle='->', color=color, lw=2))

ax[1].annotate(
        '140 keV', 
        xy=(140, -0.13),     
        xytext=(130, -0.18), 
        arrowprops=dict(arrowstyle='->', color=color, lw=2))

ax[2].annotate(
        '140 keV', 
        xy=(140, 0.8),     
        xytext=(130, 0.9), 
        arrowprops=dict(arrowstyle='->', color=color, lw=2))

    
plt.tight_layout()
plt.subplots_adjust(hspace=0.05)  
fig.savefig('plots/a_b_g.png',dpi=300)

In [99]:
fig,ax = plt.subplots()
ax.plot(energy,beta)
fontsize=16
ax.set_xlabel('Energia [keV]',fontsize=fontsize)
ax.set_ylabel(r'$\alpha$',fontsize=fontsize)


Text(0, 0.5, '$\\alpha$')

In [21]:
import pandas as pd

In [66]:
df=pd.read_csv('data3.txt',sep='\t')
df.columns

Index(['T', 'α3', 'β3', 'γ3', 'χ2_3', 'α4', 'β4', 'γ4', 'χ2_4'], dtype='object')

In [69]:
df_subset = df[['T', 'α3', 'β3', 'γ3', 'χ2_3']]
material_column = ['Acero'] * len(df_subset)
df_subset.insert(0, 'Material', material_column)

In [70]:
print(df_subset.to_csv(index=False))

Material,T,α3,β3,γ3,χ2_3
Acero,10,35.068,80.286,0.1718,0.2623
Acero,15,42.896,-14.135,1.1513,0.0
Acero,20,19.721,-9.6886,2.7227,0.00058279
Acero,25,10.357,-4.9682,3.419,0.004177
Acero,30,6.1395,-2.0142,2.3201,3.8275e-05
Acero,35,3.9145,-1.4049,2.9659,0.02918
Acero,40,2.6645,-0.60655,1.7984,7.1274e-05
Acero,45,1.8981,-0.40604,1.7432,0.021919
Acero,50,1.4112,-0.22685,1.1963,0.0011541
Acero,55,1.08,-0.17254,1.2441,0.0041784
Acero,60,0.85248,-0.13327,1.1595,0.017991
Acero,65,0.68752,-0.10514,1.157,0.022259
Acero,70,0.56286,-0.089759,1.3976,0.010666
Acero,75,0.47528,-0.075517,1.1653,0.017669
Acero,80,0.40599,-0.06599,1.1244,1.923
Acero,85,0.40988,-0.065411,0.88713,0.038793
Acero,90,0.30668,-0.052336,1.2367,0.0088909
Acero,95,0.27828,-0.050199,0.83917,0.010167
Acero,100,0.24895,-0.047754,0.93083,0.015697
Acero,105,0.22397,-0.044848,1.0666,0.0069165
Acero,110,0.20743,-0.043758,0.8825,0.013961
Acero,115,0.18986,-0.041533,0.96743,0.0070746
Acero,120,0.1761,-0.040423,0.9852,0.004907
Acero,125,0.